<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Evaluating a Multi-Object Tracking Model on MOT Challenge

This notebook is to evaluate [FairMOT](https://github.com/ifzhang/FairMOT) with [MOT Challenge dataset](https://motchallenge.net/). It hosts the most common benchmarking datasets for pedestrian MOT. Different datasets exist: MOT15, MOT16/17, MOT 19/20. These datasets contain many video sequences, with different tracking difficulty levels, with annotated ground-truth. Detections are also provided for optional use by the participating tracking algorithms.

The goals of this notebook are:
* Show the advantages of FairMOT on standardized metrics
* Easy to compare with other state of the art MOT trackers
* Re-produce the results on public dataset - MOT Challenge, contributes to larger research community

## 00 Initialization

In [1]:
import sys

sys.path.append("../../")

import os
import os.path as osp
import time
from urllib.parse import urljoin
from ipywidgets import Video
import matplotlib.pyplot as plt
import torch
import torchvision

from utils_cv.tracking.data import Urls
from utils_cv.tracking.dataset import TrackingDataset
from utils_cv.tracking.model import TrackingLearner, write_video

from utils_cv.common.data import data_path, download, unzip_url
from utils_cv.common.gpu import which_processor, is_windows

# Change matplotlib backend so that plots are shown for windows
if is_windows():
    plt.switch_backend("TkAgg")

print(f"TorchVision: {torchvision.__version__}")
which_processor()

TorchVision: 0.4.0a0+6b959ee
Torch is using GPU: Tesla K80


This shows your machine's GPUs (if it has any) and the computing device `torch/torchvision` is using.

In [2]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Next, set some model runtime parameters. We evaluate the FairMOT model on MOT17 or MOT16.

In [3]:
CONF_THRES = 0.3
TRACK_BUFFER = 300
IM_SIZE = (1080, 1920)

# Evaluate on MOT17 or MOT16
MOT16_or_17 = "MOT17"
# Downloaded MOT Challendage data path
MOT_SAVED_PATH = "../../data/"
RESEULT_ROOT = "./results"
EXP_NAME = "MOT_val_all_dla34"

FT_MODEL = "./models/all_dla34.pth"

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using torch device: {device}")

Using torch device: cuda


## 01  Pretrained Model Inference

Download [MOT17](https://motchallenge.net/data/MOT17.zip) or [MOT16](https://motchallenge.net/data/MOT16.zip) data to the path `MOT_SAVED_PATH`. The MOT17 dataset is around 5G and MOT16 is around 2G. May take some time to download.

In [4]:
base = "https://motchallenge.net/data/"
if MOT16_or_17 == "MOT17":
    mot_path = urljoin(base, "MOT17.zip")
    MOT_SAVED_PATH = os.path.join(MOT_SAVED_PATH, "MOT17", "train")
    seqs_str = '''MOT17-02-SDP
                      MOT17-04-SDP
                      MOT17-05-SDP
                      MOT17-09-SDP
                      MOT17-10-SDP
                      MOT17-11-SDP
                      MOT17-13-SDP'''
elif MOT16_or_17 == "MOT16":
    mot_path = urljoin(base, "MOT16.zip")
    MOT_SAVED_PATH = os.path.join(MOT_SAVED_PATH, "MOT16", "train")
    seqs_str = '''MOT16-02
                      MOT16-04
                      MOT16-05
                      MOT16-09
                      MOT16-10
                      MOT16-11
                      MOT16-13'''
unzip_url(mot_path, dest=MOT_SAVED_PATH, exist_ok=True)
print(MOT_SAVED_PATH)

../../data/MOT17/train


Initialize and load the model. We use the pre-trained baseline FairMOT model - `all_dla34.pth`, which can be downloaded [here](https://drive.google.com/file/d/1udpOPum8fJdoEQm6n0jsIgMMViOMFinu/view). Please upload and save `all_dla34.pth` to `FT_MODEL` path.

In [5]:
tracker = TrackingLearner(None, FT_MODEL)
print(f"Model: {type(tracker.model)}")

Model: <class 'utils_cv.tracking.references.fairmot.models.networks.pose_dla_dcn.DLASeg'>


## 02 Evaluate

`eval_mot` function calls the prediction function, saves the tracking results to txt file and provides the evaluation results with [motmetrics](https://github.com/cheind/py-motmetrics) format.

In [6]:
def eval_mot(conf_thres, track_buffer, im_size, data_root, seqs, result_root, exp_name, run_eval=False):
    eval_result = os.path.join(result_root, exp_name)
    if not osp.exists(eval_result):
        os.makedirs(eval_result)
    accs = []
    n_frame = 0
    timer_avgs, timer_calls = [], []
    for seq in seqs:
        print('start seq: {}'.format(seq))
        im_or_video_path = osp.join(data_root, seq, 'img1')
        result_filename = '{}.txt'.format(seq)
        result_file_path = os.path.join(result_root, exp_name, result_filename)
        meta_info = open(os.path.join(data_root, seq, 'seqinfo.ini')).read()
        frame_rate = int(meta_info[meta_info.find('frameRate') + 10:meta_info.find('\nseqLength')])
        # If prediction results file doesn't exsit or run_eval = True, will re-run the inference.
        if not osp.exists(result_file_path) or run_eval:
            # Run tracking.
            eval_results = tracker.predict(
                im_or_video_path,
                conf_thres,
                track_buffer,
                im_size,
                frame_rate)
            result_file_path = tracker.savetxt_results(eval_results, exp_name, result_root, result_filename, if_tmp=False)
            print("Re-run the inference, saved in path ", result_file_path)
        print("Saved tracking result txt path: ", result_file_path)
        # eval
        print('Evaluate seq: {}'.format(seq))
        mot_accumulator = tracker.evaluate_mot(data_root, seq, result_file_path)
        accs.append(mot_accumulator)
    strsummary = tracker.mot_summary(accs, seqs)
    return strsummary

We use training dataset from MOT17 or MOT16 to evaluate with the ground truth. The following evaluation results on MOT Challenge training set are comparable with the [FairMOT paper](https://arxiv.org/abs/2004.01888) reported on MOT Challenge test set:

| Dataset | MOTA   | IDF1 | IDS | MT | ML | FPS |
|------|------|------|------|------|------|------|
|   MOT16  | 68.7| 70.4| 953| 39.5%| 19.0%| 25.9|
|   MOT17  | 67.5| 69.8| 2868| 37.7%| 20.8%| 25.9|

For evaluating on testing dataset, you can get the txt prediction results in this section and submit to the [MOT Challenge](https://motchallenge.net/) evaluation server to obtain the results.

Evaluate on MOT17.

In [7]:
seqs = [seq.strip() for seq in seqs_str.split()]

# run_eval: if need to re-run the prediction
strsummary = eval_mot(conf_thres=CONF_THRES,
             track_buffer=TRACK_BUFFER,
             im_size=IM_SIZE,
             data_root=MOT_SAVED_PATH,
             seqs=seqs,
             result_root = RESEULT_ROOT,
             exp_name = EXP_NAME,
             run_eval = False)
print(strsummary)

start seq: MOT17-02-SDP
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT17-02-SDP.txt
Evaluate seq: MOT17-02-SDP
start seq: MOT17-04-SDP
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT17-04-SDP.txt
Evaluate seq: MOT17-04-SDP
start seq: MOT17-05-SDP
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT17-05-SDP.txt
Evaluate seq: MOT17-05-SDP
start seq: MOT17-09-SDP
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT17-09-SDP.txt
Evaluate seq: MOT17-09-SDP
start seq: MOT17-10-SDP
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT17-10-SDP.txt
Evaluate seq: MOT17-10-SDP
start seq: MOT17-11-SDP
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT17-11-SDP.txt
Evaluate seq: MOT17-11-SDP
start seq: MOT17-13-SDP
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT17-13-SDP.txt
Evaluate seq: MOT17-13-SDP
              IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML    FP    FN IDs    FM  MOTA  MOTP IDt I

Change MOT16_or_17 = "MOT16". Re-run the notebook and evaluate on MOT16.

In [9]:
seqs = [seq.strip() for seq in seqs_str.split()]

# run_eval: if need to re-run the prediction
strsummary = eval_mot(conf_thres=CONF_THRES,
             track_buffer=TRACK_BUFFER,
             im_size=IM_SIZE,
             data_root=MOT_SAVED_PATH,
             seqs=seqs,
             result_root = RESEULT_ROOT,
             exp_name = EXP_NAME,
             run_eval = False)
print(strsummary)

start seq: MOT16-02
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT16-02.txt
Evaluate seq: MOT16-02
start seq: MOT16-04
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT16-04.txt
Evaluate seq: MOT16-04
start seq: MOT16-05
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT16-05.txt
Evaluate seq: MOT16-05
start seq: MOT16-09
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT16-09.txt
Evaluate seq: MOT16-09
start seq: MOT16-10
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT16-10.txt
Evaluate seq: MOT16-10
start seq: MOT16-11
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT16-11.txt
Evaluate seq: MOT16-11
start seq: MOT16-13
Saved tracking result txt path:  ./results/MOT_val_all_dla34/MOT16-13.txt
Evaluate seq: MOT16-13
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML    FP    FN IDs    FM  MOTA  MOTP IDt IDa IDm
MOT16-02 60.1% 67.3% 54.3% 73.6% 91.3%  54  26  24  4  1252  4702 239   590 65.3%